In [1]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop

from tensorflow.keras import layers
import numpy as np
import tensorflow as tf
from tensorflow import keras


from dataset_manager import *
from predict import *

train_dataset, test_dataset = get_dataset()
d_list = list(test_dataset)

X_train, Y_train = get_X_Y(train_dataset)

X_test, Y_test = get_X_Y(test_dataset)

#X = []
#for i in range(0, len(Y_train) - 1):
#    X.append([Y_train[i], X_train[i]])




In [2]:
from sklearn.datasets import make_regression
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import SGD
from matplotlib import pyplot
from dataset_manager import *
from predict import *

train_dataset, test_dataset = get_dataset()
X_train, Y_train = get_X_Y_vectorized_int(train_dataset)
X_test, Y_test = get_X_Y_vectorized_int(test_dataset)

X_train, Y_train = shuffle_X_Y(X_train, Y_train)
X_test, Y_test = shuffle_X_Y(X_test, Y_test)


model = Sequential()
#model.add(Flatten(input_shape=(len(X_train), 17)))
model.add(Dense(17))
for _ in range(6):
    model.add(Dense(128, activation='sigmoid'))
model.add(Dense(42))

model.compile(loss='mean_squared_error', optimizer='adam')


model.fit(np.array(X_train), np.array(Y_train), epochs=10, batch_size=64)
model.summary()

Epoch 1/10
3938/3938 [==============================] - 5s 1ms/step - loss: 0.0257
Epoch 2/10
3938/3938 [==============================] - 5s 1ms/step - loss: 0.0173
Epoch 3/10
3938/3938 [==============================] - 5s 1ms/step - loss: 0.0143
Epoch 4/10
3938/3938 [==============================] - 5s 1ms/step - loss: 0.0135
Epoch 5/10
3938/3938 [==============================] - 5s 1ms/step - loss: 0.0132
Epoch 6/10
3938/3938 [==============================] - 5s 1ms/step - loss: 0.0130
Epoch 7/10
3938/3938 [==============================] - 5s 1ms/step - loss: 0.0128
Epoch 8/10
3938/3938 [==============================] - 5s 1ms/step - loss: 0.0126
Epoch 9/10
3938/3938 [==============================] - 5s 1ms/step - loss: 0.0126
Epoch 10/10
3938/3938 [==============================] - 5s 1ms/step - loss: 0.0124
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)    

In [3]:
test_acc = model.evaluate(np.array(X_test), np.array(Y_test), verbose=2)

print('\nTest loss:', test_acc)

3113/3113 - 1s - loss: 0.0118

Test loss: 0.011798979714512825


In [14]:
def get_prediction_list(model, pic="tohack/pics_LOGINMDP"):
    """
    The model is like "model = LinearRegression()" (or other algorithms)
    """

    x_pred, info = get_pics_from_file(f"../{pic}.bin")
    
    res = []

    preds_brut = model.predict(np.array(x_pred))
    for pred in preds_brut:
        itemindex = np.where(pred==max(pred))[0][0]
        prediction = d_list[itemindex]
        if "tohack" in pic or prediction != 'NOKEY':
            res.append(prediction)
    return res


def get_creds(prediction_list):
    for i_k in range(len(prediction_list)):
        if prediction_list[i_k] == 'CTRL' and i_k + 2 < len(prediction_list) and prediction_list[i_k + 2] == 'SUPPR':
            print('#####################')
            print(prediction_list[i_k:i_k + 42])


In [16]:
def compute_accuracy_key(key):
    res = get_prediction_list(model, pic=f"data/pics_{key}")
    return (key, len(res), res.count(key), res.count(key) / len(res) * 100)
    
def compute_accuracy():
    accuracy = {}
    for key in d_list:
        accuracy[key] = compute_accuracy_key(key)[-1]
        
    return accuracy
    
accuracy = compute_accuracy()
for key, acc in accuracy.items():
    print(key, acc)

print("accuracy: ", sum(accuracy.values()) / 42)


I 84.41490747980193
D 63.86543873113328
2 63.86473429951691
CTRL 97.47154225848695
E 28.669288342001774
J 79.68086475357097
M 65.11627906976744
8 0.0
R 38.58915741345526
Z 57.790515893694625
0 29.475755120935077
SPACE 13.31793687451886
5 42.55268199233716
W 64.02569593147751
SUPPR 88.61870142892955
Y 9.333168071393159
Q 9.146585008653974
L 89.7429435483871
9 48.27801347641627
B 45.22903268145426
O 82.238082413143
7 52.36082598719961
NOKEY 0.0
P 48.22905894519131
U 0.0
X 41.90077704722056
ENTER 85.27433099117955
3 0.0
SHIFT 98.6605213884488
4 59.65383191410588
6 51.03911191428941
F 46.80187207488299
H 59.171597633136095
S 69.29143411793254
T 38.17136285221391
A 36.710875331564985
1 0.0
C 85.01288659793815
V 87.03455011598096
K 81.64251207729468
G 49.908854166666664
N 85.39674655407923
accuracy:  51.849582726152384


In [6]:
res = get_prediction_list(model)
print(res)

['Z', 'R', 'A', '7', 'K', 'Z', 'R', '4', '7', 'Z', 'E', 'T', '4', 'Z', 'E', 'A', 'R', 'CTRL', '6', 'P', 'CTRL', '7', 'Z', 'K', '4', 'P', 'R', '2', '4', 'R', 'T', 'Z', 'T', '4', 'R', 'R', '4', '4', '7', 'H', 'CTRL', 'H', '4', '2', '6', 'H', 'E', '7', 'CTRL', 'P', 'R', 'CTRL', 'A', 'E', 'CTRL', 'SUPPR', 'A', '6', 'E', 'CTRL', 'Z', 'H', 'Z', '7', 'R', 'A', '7', '5', 'Z', 'SUPPR', '5', 'A', 'CTRL', 'SHIFT', '4', '2', 'H', 'CTRL', 'CTRL', 'T', '5', '4', 'CTRL', '4', 'T', '4', '2', 'CTRL', '4', 'P', 'Z', 'T', 'P', 'CTRL', 'CTRL', 'E', 'CTRL', 'Z', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTR

In [7]:
get_creds(res)

#####################
['CTRL', 'CTRL', 'SUPPR', 'CTRL', '4', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', '0', 'SUPPR', 'SUPPR', 'SUPPR', '0', 'SUPPR', 'SUPPR', 'SUPPR', '2', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR']
#####################
['CTRL', 'CTRL', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', '0', 'SUPPR', 'SUPPR', 'SUPPR', '0', 'SUPPR', 'SUPPR', 'SUPPR', '2', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR']
#####################
['CTRL', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', '0', 'SUPPR', 'SUPPR', 'SUPPR', '0', 'SUPPR', 'SUPPR', 'SUPPR', '2', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 